In [1]:
import pandas as pd
import arcgis

# Load Data into a SpatialDataFrame

Load the data into a Pandas DataFrame from a CSV file.

In [19]:
df = pd.read_csv('./store_locations.csv', index_col='OBJECTID')
df.head()

,LOCNUM,SALESVOL,X,Y
OBJECTID,,,,
1,666990510,35495,-121.8430,36.6210
2,653371815,35495,-121.8112,36.6676
3,423468472,35495,-121.9651,36.9753
4,511743478,35495,-121.7740,36.9154
5,404459478,52059,-122.0362,37.3231


While the coordinates for each store are contained in an X (longitude) and Y (latitude) field, the data is not yet able to be recognized spatially. We need to create a point geometry for each location in a new field so the data will be recognized as spatial. Once this is done, we also can get rid of the explicity X and Y fields, since the location is now stored in the SHAPE field.

In [20]:
df['SHAPE'] = df.apply(lambda row: arcgis.geometry.Point({'x': row.X, 'y': row.Y, 'spatialReference': {'wkid': 4326}}), axis=1)
df.head()

,LOCNUM,SALESVOL,X,Y,SHAPE
OBJECTID,,,,,
1,666990510,35495,-121.8430,36.6210,"{'x': -121.843, 'y': 36.621, 'spatialReference..."
2,653371815,35495,-121.8112,36.6676,"{'x': -121.8112, 'y': 36.6676, 'spatialReferen..."
3,423468472,35495,-121.9651,36.9753,"{'x': -121.9651, 'y': 36.9753, 'spatialReferen..."
4,511743478,35495,-121.7740,36.9154,"{'x': -121.774, 'y': 36.9154, 'spatialReferenc..."
5,404459478,52059,-122.0362,37.3231,"{'x': -122.0362, 'y': 37.3231, 'spatialReferen..."


Now, with the location data stored as point geometry, we can create a SpatialDataFrame with the store locations so the data will now be recognized as a spatial table.

In [21]:
sdf = arcgis.features.SpatialDataFrame(df)
sdf.set_geometry(col='SHAPE')
sdf.reset_index(inplace=True)
sdf = sdf.drop(['X', 'Y'], axis=1)
sdf.head()

,OBJECTID,LOCNUM,SALESVOL,SHAPE
0,1,666990510,35495,"{'x': -121.843, 'y': 36.621, 'spatialReference..."
1,2,653371815,35495,"{'x': -121.8112, 'y': 36.6676, 'spatialReferen..."
2,3,423468472,35495,"{'x': -121.9651, 'y': 36.9753, 'spatialReferen..."
3,4,511743478,35495,"{'x': -121.774, 'y': 36.9154, 'spatialReferenc..."
4,5,404459478,52059,"{'x': -122.0362, 'y': 37.3231, 'spatialReferen..."


In [22]:
# get a subset to test with, just the first five records
sdf = sdf[:5]
sdf

,OBJECTID,LOCNUM,SALESVOL,SHAPE
0,1,666990510,35495,"{'x': -121.843, 'y': 36.621, 'spatialReference..."
1,2,653371815,35495,"{'x': -121.8112, 'y': 36.6676, 'spatialReferen..."
2,3,423468472,35495,"{'x': -121.9651, 'y': 36.9753, 'spatialReferen..."
3,4,511743478,35495,"{'x': -121.774, 'y': 36.9154, 'spatialReferenc..."
4,5,404459478,52059,"{'x': -122.0362, 'y': 37.3231, 'spatialReferen..."


Convert the SpatailDataFrame to a FeatureSet to use as input for creating drive time trade areas.

__NOTE:__ As of 18Aug2017, this will not work unless you have access to the development reposititory, since there was a bug in the `to_featureset` method, which was recently fixed.

In [26]:
fs = sdf.to_featureset()  # for now, just testing with first five records
fs

{"fields": [], "geometryType": "esriGeometryPoint", "features": [{"attributes": {"SALESVOL": 35495, "OBJECTID": 1, "LOCNUM": 666990510}, "geometry": {"x": -121.84299999999992, "y": 36.62100000000007, "spatialReference": {"wkid": 4326}}}, {"attributes": {"SALESVOL": 35495, "OBJECTID": 2, "LOCNUM": 653371815}, "geometry": {"x": -121.84299999999992, "y": 36.62100000000007, "spatialReference": {"wkid": 4326}}}, {"attributes": {"SALESVOL": 35495, "OBJECTID": 3, "LOCNUM": 423468472}, "geometry": {"x": -121.84299999999992, "y": 36.62100000000007, "spatialReference": {"wkid": 4326}}}, {"attributes": {"SALESVOL": 35495, "OBJECTID": 4, "LOCNUM": 511743478}, "geometry": {"x": -121.84299999999992, "y": 36.62100000000007, "spatialReference": {"wkid": 4326}}}, {"attributes": {"SALESVOL": 52059, "OBJECTID": 5, "LOCNUM": 404459478}, "geometry": {"x": -121.84299999999992, "y": 36.62100000000007, "spatialReference": {"wkid": 4326}}}]}

# Create GIS Instance

In [27]:
from getpass import getpass

gis_coldbrew = arcgis.gis.GIS(
    url='http://portal.coldbrew.esri.com/portal',
    username='headless', 
    password=getpass('Please enter the headless password: ')
)

Please enter the headless password: ········


In [28]:
service_area_layer = arcgis.network.ServiceAreaLayer(
    url=gis_coldbrew.properties.helperServices.serviceArea.url, 
    gis=gis_coldbrew
)

In [29]:
travel_modes = service_area_layer.retrieve_travel_modes()
travel_mode_drive = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Driving Time'][0]

In [30]:
result = service_area_layer.solve_service_area(
    facilities=fs, 
    travel_mode=travel_mode_drive, 
    default_breaks=[8]
)

In [32]:
type(result)

dict

In [33]:
result

{'messages': [{'description': 'Input field [SALESVOL] was not mapped to a field in the network analysis class "Facilities".',
   'type': 50},
  {'description': 'Input field [LOCNUM] was not mapped to a field in the network analysis class "Facilities".',
   'type': 50}],
 'saPolygons': {'features': [{'attributes': {'FacilityID': 2,
     'FromBreak': 0,
     'Name': 'Location 2 : 0 - 8',
     'ObjectID': 1,
     'Shape_Area': 0.002164947423158558,
     'Shape_Length': 1.5493428284911819,
     'ToBreak': 8},
    'geometry': {'rings': [[[-121.86070632899998, 36.58175468400003],
       [-121.86250305199997, 36.58152961700006],
       [-121.86250305199997, 36.57816124000004],
       [-121.86272811899994, 36.57973289500006],
       [-121.86272811899994, 36.57951164200006],
       [-121.86362648, 36.57995796200004],
       [-121.86362648, 36.58040809600004],
       [-121.86272811899994, 36.58018302900007],
       [-121.86295318599997, 36.58175468400003],
       [-121.86632156399997, 36.5835514